In [1]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

import operator
from gensim.models.keyedvectors import FastTextKeyedVectors

from text_processing_udpipe_w2v import get_text_map

from tqdm import tqdm

import statistics 

import json

C:\Users\Asus\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
dp_300 = pd.read_csv("3000.csv", sep = ",")
dp_300.head()

,Unnamed: 0,texts_3000,same_word_list_3000,percent_3000
0,0,Указом президента России Бориса Ельцина внесен...,"{'отмечать', 'изменение', 'иностранный', 'роль...","71,25"
1,1,В Харькове неизвестные изуродовали лицо депута...,"{'регион', 'серьезный', 'агентство', 'являться...","72,22222222"
2,2,"Руководитель следствия по делу ""Аэрофлота"" ...","{'ставить', 'ход', 'фирма', 'отказываться', 'с...",74
3,3,Постоянный экономический рост позволит Китаю к...,"{'внутренний', 'момент', 'сфера', 'слово', 'мн...","72,60273973"
4,4,В одном из ночных клубов Хмельницкого группа п...,"{'отмечать', 'повод', 'смерть', 'свидетель', '...","70,68965517"


In [3]:
word_dp = pd.read_csv("word_db.csv", sep = ",", header=None)
word_dp.head()

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
0,0.025718,-0.004776,0.031789,0.018760,-0.015405,-0.078441,-0.002449,-0.008062,-0.001146,0.010007,...,0.017842,-0.041569,0.077341,0.012496,0.049400,-0.037332,0.001658,0.010653,-0.036526,-0.000000
1,0.018746,0.003426,0.020213,0.056091,-0.006876,-0.030144,0.021508,-0.038640,-0.008778,0.014135,...,0.000866,-0.069830,0.107995,0.041295,0.074002,-0.027547,-0.023377,-0.026141,-0.028869,-0.062423
2,0.030954,0.012660,-0.013010,0.058922,-0.015895,-0.004156,0.043479,-0.097470,0.013334,-0.000223,...,-0.012979,-0.054481,0.129445,0.047809,0.064443,-0.009311,-0.068905,-0.063734,-0.017192,-0.050362
3,0.029982,0.046834,-0.016939,0.034865,-0.004499,-0.025411,0.057243,-0.126531,0.015994,-0.012146,...,-0.063004,0.012827,0.101894,0.074595,0.044913,-0.022268,-0.048629,-0.074751,0.018179,-0.035168
4,0.023059,0.052791,-0.031716,0.051026,-0.040494,-0.007667,0.052596,-0.087467,-0.004326,0.018472,...,-0.052820,0.033877,0.062752,0.100702,0.030436,0.018350,-0.031551,-0.050780,0.061213,-0.059490


In [4]:
trigramms =[]
with open ("trigramm_db.txt", "r", encoding = "utf-8") as f:
    for trig in f.readlines():
        trigramms.append(trig[:-1])

In [ ]:
fasttext = FastTextKeyedVectors.load("D:/fasttext_word2vec/araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model")

In [ ]:
potential_recommended_text_map = get_text_map("text_8.txt")

In [6]:
def trigramms_recommendation_store (rec_text_map, marked_trigramms_db, trigrmms_list):
    sentences_map = rec_text_map['sentences_map']
    rec_text_dict = {}#key = trigr from recommended text value = list of similar trigramms from marked db
    for sentence_ind in range(len(sentences_map)):
        for word in sentences_map[sentence_ind]['sentence_words']:
            text_word_vec = word['lex_vector'].reshape(1, -1)
            text_word_dist_dict = {}
            for word_db_ind in range(len(marked_trigramms_db)):
                marked_trigramms_db_word_vec = marked_trigramms_db.iloc[word_db_ind][:-1].values.reshape(1, -1)
                similarity = cosine_similarity(text_word_vec,marked_trigramms_db_word_vec)
                text_word_dist_dict[word_db_ind] = similarity[0][0]
           
            sorted_x = sorted(text_word_dist_dict.items(), key=operator.itemgetter(1), reverse = True)
            filtered_sorted_list = filter_similar_trigramms(sorted_x)
            if not filtered_sorted_list:
                rec_text_dict[word['lex_trigram']] = None
            else:
                similar_trigrams= []
                for similar_el in filtered_sorted_list:
                    similar_trigrams.append([similar_el, trigrmms_list[similar_el[0]],abs(list(marked_trigramms_db.iloc[similar_el[0]])[-1])])#None - в зависимости от 
                rec_text_dict[word['lex_trigram']] = similar_trigrams
    return rec_text_dict
#ПОЛСЕ ТОГО КАК МЫ ПОЛУЧИЛИ ОТВЕТЫ НА ВОПРОСЫ ФОРМИРУЕТСЯ ДАТАСЕТ С ЦЕЛЕВОЙ ПЕРЕМЕННОЙ
#СООТВЕТТСВЕННО МОЖНО ПРОСТО ПРОЙТИСЬ ПО ВСЕМУ НАКОПЛЕННОМУ СЛОВАРЮ И ПРОСТО ПРОСТАВИТЬ МИНУСЫ ТАМ ГДЕ НАДО (ПО ИНДЕКСАМ ) АДАЛЕЕ ПРОДОЛЖИТЬ ПОДСЧЕТ
#STORE_JSON  = trigramms_recommendation_store(potential_recommended_text_map,word_dp,  trigramms)

In [7]:
answer_dict = {"222" :['0-' ,'1-' ,'2+' ,'4+'],
"862":['1+','2-', '4+', '6-', '12-'],
"321":['5+' ,'0-' ,'4-' ,'1+'],
"364":['2+', '6-', '3-', '7-' ],
"502":['13-', '3+', '0-', '4+'],
"878":['3-', '0+', '4-', '1-'],
"666":['3-', '1-', '2-' ],
"92":['1+', '2-'],
"615":['1+', '2-'],
"450":['6-', '7-', '11+', '9+' ,'10+' ],
"722":['1+', '6-', '2+', '3+', '6-', '8-'],
"732":['8-', '0-', '3-', '4-', '2+', '1-'],
"611":['1+'],
"251":['5-', '4+', '2+', '1+', '3+']}
list(answer_dict.keys())

['222',
 '862',
 '321',
 '364',
 '502',
 '878',
 '666',
 '92',
 '615',
 '450',
 '722',
 '732',
 '611',
 '251']

In [ ]:
with open('./texts_json/data.json',  'w', encoding = "utf-8") as outfile:  
    json.dump(STORE_JSON, outfile, ensure_ascii=False)

In [ ]:
СОСТАВИТЬ СООТВЕТСТВИЕ (ПРЕДЛОЖЕНИЕ - ИНДЕКС ТРИГРАММЫ В БАЗЕ ДАННЫХ) И РАЗМЕЧАТЬ СООТВЕТСТВУЮЩИМ ОБРАОМ ПОСЛЕ ПОЛУЧЕНИЯ ОТВЕТОВ

In [ ]:
potential_recommended_text_map

In [8]:
for text_ind in tqdm(range(299,len(dp_300))):
    if text_ind not in list(answer_dict.keys()):
        potential_recommended_text_map = get_text_map(dp_300.iloc[text_ind]['texts_3000'], raw_text_input= True)
        STORE_JSON  = trigramms_recommendation_store(potential_recommended_text_map,word_dp,trigramms)
        save_loc = './texts_json/text_' + str(text_ind) + ".json"
        with open(save_loc,  'w', encoding = "utf-8") as outfile:  
            json.dump(STORE_JSON, outfile, ensure_ascii=False)

100%|████████████████████████████████████████████████████████████████████████████| 599/599 [22:33:06<00:00, 135.54s/it]


In [ ]:
def trigramms_recommendation (rec_text_map, marked_trigramms_db, trigrmms_list):
    sentences_map = rec_text_map['sentences_map']
    calc_text_dict = {}
    for sentence_ind in tqdm(range(len(sentences_map))):
        for word in sentences_map[sentence_ind]['sentence_words']:
            text_word_vec = word['lex_vector'].reshape(1, -1)
            text_word_dist_dict = {}
            for word_db_ind in range(len(marked_trigramms_db)):
                marked_trigramms_db_word_vec = marked_trigramms_db.iloc[word_db_ind][:-1].values.reshape(1, -1)
                similarity = cosine_similarity(text_word_vec,marked_trigramms_db_word_vec)
                text_word_dist_dict[word_db_ind] = similarity
           
            sorted_x = sorted(text_word_dist_dict.items(), key=operator.itemgetter(1), reverse = True)
            filtered_sorted_list = filter_similar_trigramms(sorted_x)
            if not filtered_sorted_list:
                calc_text_dict[word['lex_trigram']] = None
            else:
                similar_trigrams= []
                for similar_el in filtered_sorted_list:
                    similar_trigrams.append((similar_el, trigrmms_list[similar_el[0]],list(marked_trigramms_db.iloc[similar_el[0]])[-1] ))
                calc_text_dict[word['lex_trigram']] = similar_trigrams
                #=====ЗДЕСЬ ПР ПОДГРУЗКЕ НАДО БУДЕТ РАЗМЕТИТЬ ЦЕЛЕВУЮ ПЕРЕМЕННУЮВ СООТВТЕТСВИИ С КОРРЕКТНОСТЬЮ ОТВЕТА И ДАЛЕЕ ПЛЯСАТЬ
                
    vocab_recommendation_dict = {}
    for key in calc_text_dict.keys():
        print(key , "===>", calc_text_dict[key],'\n')
        understdanding_prob_list = []
        if (calc_text_dict[key]):
            for trgrm in calc_text_dict[key]:
                understdanding_prob = trgrm[2]
                understdanding_prob_list.append(understdanding_prob)
            understdanding_prob_mean = statistics.mean(understdanding_prob_list) 
            vocab_recommendation_dict[key] = understdanding_prob
    positive_understanding_prob = 0
    whole_understading_prob = 0 
    for key in vocab_recommendation_dict.keys():
        print(key , "===>", vocab_recommendation_dict[key],'\n')
        if (vocab_recommendation_dict[key] > 0):
            positive_understanding_prob += vocab_recommendation_dict[key]
            whole_understading_prob += abs(vocab_recommendation_dict[key])
        elif(vocab_recommendation_dict[key] < 0):
            whole_understading_prob += abs(vocab_recommendation_dict[key])
    understanding_prob = positive_understanding_prob/whole_understading_prob
    print("understanding_prob",understanding_prob)
        
trigramms_recommendation(potential_recommended_text_map,word_dp,  trigramms)

In [ ]:
sent_dp = pd.read_csv("sentence_db.csv", sep = ",", header=None)
sent_dp.head()

In [ ]:
text_db = pd.read_csv("text_db.csv", sep = ",", header=None)
text_db.head()

In [ ]:
def sent_text_features_recommendation (rec_text_map, marked_sentences_db, marked_text_db):
    rec_text_features = []
    rec_text_features.append(rec_text_map['lix'])
    rec_text_features.append(rec_text_map['ttr'])
    rec_text_features.extend(rec_text_map['sent_properties'])
    rec_text_vec = np.array(rec_text_features).reshape(1, -1)
    #print(rec_text_vec)
    text_similarity_dict = {}
    for text_similarity_ind in range(len(marked_text_db)):
        text_vec = marked_text_db.iloc[text_similarity_ind][:-1].values.reshape(1, -1)
        #print(text_vec)
        similarity = cosine_similarity(rec_text_vec,text_vec)
        #print(similarity)
        text_similarity_dict[text_similarity_ind] = similarity
    sorted_text_feat_dict = sorted(text_similarity_dict.items(), key=operator.itemgetter(1), reverse = True)
    text_feeat_mean_dict = []
    for text_el in sorted_text_feat_dict[:min(3, len(sorted_text_feat_dict))]:
        understandng = list(marked_text_db.iloc[text_el[0]])[-1]
        text_feeat_mean_dict.append(understandng)
    text_feeat_mean = statistics.mean(text_feeat_mean_dict)
    
    #print(sorted_text_feat_dict)
    print("text_understanding_prob" , text_feeat_mean)
    
    sentences_understanding = []
    for sentence in rec_text_map['sentences_map']:
        rec_sent_feat = []
        rec_sent_feat.append(sentence['spec_sentence_features']['negation'])
        rec_sent_feat.append(sentence['spec_sentence_features']['coreference'])
        rec_sent_feat.append(sentence['spec_sentence_features']['vozvr_verb'])
        rec_sent_feat.append(sentence['spec_sentence_features']['prich'])
        rec_sent_feat.append(sentence['spec_sentence_features']['deepr'])
        rec_sent_feat.append(sentence['spec_sentence_features']['case_complexity'])
        rec_sent_feat.append(sentence['spec_sentence_features']['mean_depend_length'])
        rec_sent_vec = np.array(rec_sent_feat).reshape(1, -1)
        sent_similarity_dict = {}
        for sent_ind in range(len(marked_sentences_db)):
            sent_vec = marked_sentences_db.iloc[sent_ind][:-1].values.reshape(1, -1)
            similarity = cosine_similarity(rec_sent_vec,sent_vec)
            sent_similarity_dict[sent_ind] = similarity
        sorted_sent_similarity_dict = sorted(sent_similarity_dict.items(), key=operator.itemgetter(1), reverse = True)
        sent_similarity_list = []
        for sim_sent in sorted_sent_similarity_dict[:3]:
            #print(sim_sent)
            sent_undestanding_mark = list(marked_sentences_db.iloc[sim_sent[0]])[-1]
            sent_similarity_list.append(sent_undestanding_mark)
        #print(sent_similarity_list)
        sent_similarity_mean = statistics.mean(sent_similarity_list)
        #print(sent_similarity_mean)
        sentences_understanding.append(sent_similarity_mean)
    sentences_understanding_mean = statistics.mean(sentences_understanding)
    print("sentences_understanding_mean", sentences_understanding_mean)
             
sent_text_features_recommendation(potential_recommended_text_map, marked_text_db = text_db,marked_sentences_db =  sent_dp)    

In [ ]:
potential_recommended_text_map